In [ ]:
import unittest
import sqlite3
from datetime import datetime, timedelta
from db_controller import *

class TestDatabaseController(unittest.TestCase):
    def setUp(self):
        # Create an in-memory SQLite database
        self.conn = sqlite3.connect(':memory:')
        self.cursor = self.conn.cursor()
        self.initialize_schema()
        
        self.db_controller = DatabaseController()
        self.db_controller._conn = self.conn
        self.db_controller._cursor = self.cursor

    def tearDown(self):
        # Close the in-memory database
        self.conn.close()

    def initialize_schema(self):
        # Create tables needed for the tests
        self.cursor.executescript("""
            CREATE TABLE Book (
                ISBN TEXT PRIMARY KEY,
                Title TEXT,
                Authors TEXT,
                Description TEXT,
                Genre TEXT,
                availability INTEGER
            );
            CREATE TABLE Users (
                user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                password TEXT,
                favourite_genre TEXT,
                fine REAL
            );
            CREATE TABLE Comment (
                comment_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                book_isbn TEXT,
                comment_text TEXT,
                comment_date DATETIME,
                FOREIGN KEY(user_id) REFERENCES Users(user_id),
                FOREIGN KEY(book_isbn) REFERENCES Book(isbn)
            );
            CREATE TABLE Notification (
                notification_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                message TEXT,
                notification_date DATETIME,
                FOREIGN KEY(user_id) REFERENCES Users(user_id)
            );
            CREATE TABLE ReadingList (
                reading_list_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                book_isbn TEXT,
                FOREIGN KEY(user_id) REFERENCES Users(user_id),
                FOREIGN KEY(book_isbn) REFERENCES Book(isbn)
            );
            CREATE TABLE Reservation (
                reservation_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                book_isbn TEXT,
                reservation_date DATETIME,
                due_date DATETIME,
                status TEXT,
                FOREIGN KEY(user_id) REFERENCES Users(user_id),
                FOREIGN KEY(book_isbn) REFERENCES Book(isbn)
            );
        """)
        self.conn.commit()

    def test_add_user(self):
        result = self.db_controller.add_user("Alice", "password123", "Fiction")
        self.assertTrue(result)
        self.cursor.execute("SELECT * FROM Users WHERE name = ?", ("Alice",))
        user = self.cursor.fetchone()
        self.assertIsNotNone(user)
        self.assertEqual(user[1], "Alice")

    def test_add_book(self):
        result = self.db_controller.add_book("1234567890", "Test Book", "Author Name", "Description", "Fiction", 1)
        self.assertTrue(result)
        self.cursor.execute("SELECT * FROM Book WHERE isbn = ?", ("1234567890",))
        book = self.cursor.fetchone()
        self.assertIsNotNone(book)
        self.assertEqual(book[1], "Test Book")

    def test_search_books(self):
        # Add sample books
        self.db_controller.add_book("111", "Python Programming", "Guido van Rossum", "Learn Python.", "Programming", 1)
        self.db_controller.add_book("222", "Advanced Python", "Raymond Hettinger", "Deep dive into Python.", "Programming", 1)
        # Search by title
        results = self.db_controller.search_books(title="Python")
        self.assertEqual(len(results), 2)
        # Search by author
        results = self.db_controller.search_books(author="Guido")
        self.assertEqual(len(results), 1)
        self.assertEqual(results[0][1], "Python Programming")
        # Search by genre
        results = self.db_controller.search_books(genre="Programming")
        self.assertEqual(len(results), 2)

    def test_add_comment(self):
        # Add a user and a book first
        self.db_controller.add_user("Bob", "password456", "Science")
        self.db_controller.add_book("333", "Science Book", "Author B", "About science.", "Science", 1)
        # Get user_id
        self.cursor.execute("SELECT user_id FROM Users WHERE name = ?", ("Bob",))
        user_id = self.cursor.fetchone()[0]
        # Add comment
        result = self.db_controller.add_comment(user_id, "333", "Great book!")
        self.assertTrue(result)
        self.cursor.execute("SELECT * FROM Comment WHERE user_id = ? AND book_isbn = ?", (user_id, "333"))
        comment = self.cursor.fetchone()
        self.assertIsNotNone(comment)
        self.assertEqual(comment[3], "Great book!")

    def test_add_reservation(self):
        # Add a user and a book first
        self.db_controller.add_user("Carol", "password789", "History")
        self.db_controller.add_book("444", "History Book", "Author C", "About history.", "History", 1)
        # Get user_id
        self.cursor.execute("SELECT user_id FROM Users WHERE name = ?", ("Carol",))
        user_id = self.cursor.fetchone()[0]
        # Add reservation
        due_date = datetime.now() + timedelta(days=7)
        result = self.db_controller.add_reservation(user_id, "444", due_date)
        self.assertTrue(result)
        self.cursor.execute("SELECT * FROM Reservation WHERE user_id = ? AND book_isbn = ?", (user_id, "444"))
        reservation = self.cursor.fetchone()
        self.assertIsNotNone(reservation)
        self.assertEqual(reservation[5], 'active')

    def test_update_fine(self):
        # Add a user first
        self.db_controller.add_user("Dave", "password000", "Math")
        # Get user_id
        self.cursor.execute("SELECT user_id FROM Users WHERE name = ?", ("Dave",))
        user_id = self.cursor.fetchone()[0]
        # Update fine
        result = self.db_controller.update_fine(user_id, 10.0)
        self.assertTrue(result)
        self.cursor.execute("SELECT fine FROM Users WHERE user_id = ?", (user_id,))
        fine = self.cursor.fetchone()[0]
        self.assertEqual(fine, 10.0)

    def test_get_recommendations(self):
        # Add books
        self.db_controller.add_book("555", "Fantasy Book 1", "Author D", "A fantasy tale.", "Fantasy", 1)
        self.db_controller.add_book("666", "Fantasy Book 2", "Author E", "Another fantasy tale.", "Fantasy", 1)
        self.db_controller.add_book("777", "Sci-Fi Book", "Author F", "A sci-fi adventure.", "Sci-Fi", 1)
        # Get recommendations
        recommendations = self.db_controller.get_recommendations("Fantasy")
        self.assertEqual(len(recommendations), 2)

    def test_update_book_availability(self):
        # Add a book
        self.db_controller.add_book("888", "Unavailable Book", "Author G", "Out of stock.", "Drama", 0)
        # Update availability
        result = self.db_controller.update_book_availability("888", 1)
        self.assertTrue(result)
        self.cursor.execute("SELECT availability FROM Book WHERE isbn = ?", ("888",))
        availability = self.cursor.fetchone()[0]
        self.assertEqual(availability, 1)

    def test_create_book_report(self):
        # Add a book
        self.db_controller.add_book("999", "Report Book", "Author H", "For reporting.", "Education", 1)
        # Add reservations and comments
        self.db_controller.add_user("Eve", "password111", "Education")
        self.db_controller.add_user("Frank", "password222", "Education")
        self.cursor.execute("SELECT user_id FROM Users WHERE name = ?", ("Eve",))
        user_id_eve = self.cursor.fetchone()[0]
        self.cursor.execute("SELECT user_id FROM Users WHERE name = ?", ("Frank",))
        user_id_frank = self.cursor.fetchone()[0]
        due_date = datetime.now() + timedelta(days=7)
        self.db_controller.add_reservation(user_id_eve, "999", due_date)
        self.db_controller.add_reservation(user_id_frank, "999", due_date)
        self.db_controller.add_comment(user_id_eve, "999", "Informative book.")
        self.db_controller.add_comment(user_id_frank, "999", "Well-written.")
        # Generate report
        report = self.db_controller.createBookReport("999")
        self.assertIsNotNone(report)
        # Expected counts are 2 reservations and 2 comments
        self.assertEqual(report[0][4], 2)  # Reservation count
        self.assertEqual(report[0][5], 2)  # Comment count

    def test_create_user_report(self):
        # Add a user
        self.db_controller.add_user("Grace", "password333", "Thriller")
        self.cursor.execute("SELECT user_id FROM Users WHERE name = ?", ("Grace",))
        user_id_grace = self.cursor.fetchone()[0]
        # Add books
        self.db_controller.add_book("1010", "Thriller Book 1", "Author I", "Thrilling.", "Thriller", 1)
        self.db_controller.add_book("1111", "Thriller Book 2", "Author J", "More thrills.", "Thriller", 1)
        due_date = datetime.now() + timedelta(days=7)
        # Add reservations and comments
        self.db_controller.add_reservation(user_id_grace, "1010", due_date)
        self.db_controller.add_reservation(user_id_grace, "1111", due_date)
        self.db_controller.add_comment(user_id_grace, "1010", "Exciting!")
        self.db_controller.add_comment(user_id_grace, "1111", "Couldn't put it down.")
        # Generate report
        report = self.db_controller.createUserReport(user_id_grace)
        self.assertIsNotNone(report)
        # Expected counts are 2 reservations and 2 comments
        self.assertEqual(report[0][2], 2)  # Reservation count
        self.assertEqual(report[0][3], 2)  # Comment count

if __name__ == '__main__':
    unittest.main()
